In [3]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import h5py
%matplotlib inline

mat_names = glob.glob('../project_datasets/*.mat')
# each test subject got a different file - 9 test subjects
print(mat_names)

['../project_datasets/A07T_slice.mat', '../project_datasets/A02T_slice.mat', '../project_datasets/A05T_slice.mat', '../project_datasets/A08T_slice.mat', '../project_datasets/A03T_slice.mat', '../project_datasets/A06T_slice.mat', '../project_datasets/A01T_slice.mat', '../project_datasets/A04T_slice.mat', '../project_datasets/A09T_slice.mat']


/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
matfile = h5py.File(mat_names[0], 'r')
print(matfile.keys()) #image and type
image_mat = matfile['image']
image_shape = image_mat.shape # 288 (48x6) trials across 25 electrodes for 1000 time points (250Hz*4s)
print(image_shape)

[u'image', u'type']
(288, 25, 1000)


In [5]:
type_mat = matfile['type']
type_shape = type_mat.shape
print(type_shape)
# plt.plot(type_mat[0,:288]) # gets the significant values of types
# all the 0's occur after 288, and are meaningless I think
# so the image_mat, which has shape (288, 25, 1000) should correspond
# to the first 288 entries of type_mat, so
# for a single subject, training data should be image_mat, with 288 samples, each sample has shape (25, 1000)
# and our target label matrix should be type_mat[:288] (or 287?)
nans = np.sum(np.isnan(image_mat[:,:]))
print(nans) #No NaN in the data
print(len(image_mat[0:,:]))
count = 0
# for i in range(len(image_mat[0:,:])):
#  if np.sum(np.isnan(image_mat[i:,:])):
#         pass
type_set = list(set(type_mat[0,:]))
print(type_set)

(1, 1000)
0
288
[0.0, 769.0, 770.0, 771.0, 772.0]


In [7]:
EEG_channels = 22 #from project guidelines
test_count = 50 #from project guideline, 238 for train-validation and 50 for test
validation_count = 38 # 38 points in validation set and remaining 200 points in test set

In [8]:
#setting seed
np.random.seed(seed=101)
test_picked = np.random.choice(image_shape[0], test_count, replace=False)
train_val_picked = np.setdiff1d(np.arange(image_shape[0]), test_picked)
val_picked = train_val_picked[:validation_count]
train_picked = train_val_picked[validation_count:]

In [14]:
trainval_data_X = []
training_data_X = []
validation_data_X = []
test_data_X = []

trainval_data_Y = []
training_data_Y = []
validation_data_Y = []
test_data_Y = []

for i in range(len(mat_names)):
    matfile = h5py.File(mat_names[i], 'r')
    
    trainval_data_X.append(matfile['image'][sorted(train_val_picked),:EEG_channels,:]) #(238, 22, 1000) x 9
    training_data_X.append(matfile['image'][sorted(train_picked),:EEG_channels,:]) #(200, 22, 1000) x 9
    validation_data_X.append(matfile['image'][sorted(val_picked),:EEG_channels,:]) #(38, 22, 1000) x 9
    test_data_X.append(matfile['image'][sorted(test_picked),:EEG_channels,:]) #(50, 22, 1000) x 9
    
    trainval_data_Y.append(matfile['type'][0,sorted(train_val_picked)] - type_set[1]) #(238, ) x 9
    training_data_Y.append(matfile['type'][0,sorted(train_picked)] - type_set[1]) #(200, ) x 9
    validation_data_Y.append(matfile['type'][0,sorted(val_picked)] - type_set[1]) #(38, ) x 9
    test_data_Y.append(matfile['type'][0,sorted(test_picked)] - type_set[1]) #(50, ) x 9

In [15]:
training_data_shape = training_data_X[0].shape
print(training_data_shape) #(200, 22, 1000) while test data shape is (50, 22, 1000) and validation data is (38, 22,1000)
print(training_data_Y[0].shape)

(200, 22, 1000)
(200,)


In [17]:
from functools import reduce

def remove_nan_rows_A(A, b):
    if (np.isnan(A).any() or np.isnan(b).any()):
        mask = ~np.isnan(np.sum(A,axis=(1,2))) & ~np.isnan(b[:])
        A = A[mask, :, :]
        b = b[mask]
        
    assert A.shape[0] == b.shape[0]
    return A, b


cnn_trainval_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), trainval_data_X) #(2142, 22, 1000) 
cnn_training_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), training_data_X) #(1800, 22, 1000)
cnn_validation_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), validation_data_X) #(342, 22, 1000)
cnn_test_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), test_data_X) #(450, 22, 1000)

cnn_trainval_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), trainval_data_Y) #(2142, )
cnn_training_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), training_data_Y) #(1800, )
cnn_validation_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), validation_data_Y) #(342, )
cnn_test_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), test_data_Y) #(450,)

cnn_trainval_data_X, cnn_trainval_data_Y = remove_nan_rows_A(cnn_trainval_data_X, cnn_trainval_data_Y) #(1775,22,1000)
cnn_training_data_X, cnn_training_data_Y = remove_nan_rows_A(cnn_training_data_X, cnn_training_data_Y) #(1775,22,1000)
cnn_validation_data_X, cnn_validation_data_Y = remove_nan_rows_A(cnn_validation_data_X, cnn_validation_data_Y) #(340,22,1000)
cnn_test_data_X, cnn_test_data_Y = remove_nan_rows_A(cnn_test_data_X, cnn_test_data_Y) #(443,22,1000)

In [18]:
print(cnn_training_data_X.shape)
print(cnn_validation_data_X.shape)
print(cnn_test_data_X.shape)

cnn_trainval_data_X = np.transpose(cnn_trainval_data_X, (0,2,1))
cnn_training_data_X = np.transpose(cnn_training_data_X, (0,2,1))
cnn_validation_data_X = np.transpose(cnn_validation_data_X, (0,2,1))
cnn_test_data_X = np.transpose(cnn_test_data_X, (0,2,1))

mean_list = np.mean(cnn_trainval_data_X.reshape(-1, cnn_trainval_data_X.shape[-1]), axis=0)
std_list = np.sqrt((np.var(cnn_trainval_data_X.reshape(-1, cnn_trainval_data_X.shape[-1]), axis=0)))

cnn_trainval_data_X = (cnn_trainval_data_X - mean_list)/std_list
cnn_training_data_X = (cnn_training_data_X - mean_list)/std_list
cnn_validation_data_X = (cnn_validation_data_X - mean_list)/std_list
cnn_test_data_X = (cnn_test_data_X - mean_list)/std_list

(1774, 22, 1000)
(339, 22, 1000)
(445, 22, 1000)
(1774, 22, 1000)
(339, 22, 1000)
(445, 22, 1000)


In [110]:
cnn_trainval_data_X = np.transpose(cnn_trainval_data_X, (0,2,1))
cnn_training_data_X = np.transpose(cnn_training_data_X, (0,2,1))
cnn_validation_data_X = np.transpose(cnn_validation_data_X, (0,2,1))
cnn_test_data_X = np.transpose(cnn_test_data_X, (0,2,1))

print(cnn_training_data_X.shape)
print(cnn_validation_data_X.shape)
print(cnn_test_data_X.shape)

(1774, 22, 1000)
(339, 22, 1000)
(445, 22, 1000)


In [19]:
from __future__ import print_function
import keras
from keras import backend as K
from keras.engine.topology import Layer
from keras import activations
from keras import utils
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import *

In [20]:
# the squashing function.
# we use 0.5 in stead of 1 in hinton's paper.
# if 1, the norm of vector will be zoomed out.
# if 0.5, the norm will be zoomed in while original norm is less than 0.5
# and be zoomed out while original norm is greater than 0.5.
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


# define our own softmax function instead of K.softmax
# because K.softmax can not specify axis.
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


# define the margin loss like hinge loss
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)


In [21]:
def one_hot(y_):
    # Function to encode output labels from number indexes 
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [111]:
batch_size = 128
num_classes = 4
epochs = 50
(x_train, y_train), (x_test, y_test) = (cnn_training_data_X, cnn_training_data_Y), (cnn_validation_data_X, cnn_validation_data_Y)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [112]:
print(x_train.shape)

(1774, 22, 1000)


In [121]:
# A common Conv2D model
x = Input(shape=x_train.shape[1:])

x1 = Conv1D(128, 21, padding = 'same', activation='elu')(x)
x2 = Conv1D(128, 11, padding = 'same', activation='elu')(x)
x3 = Conv1D(128, 7, padding = 'same', activation='elu')(x)
x4 = Conv1D(128, 3, padding = 'same', activation='elu')(x)
x5 = Conv1D(128, 1, padding = 'same', activation='elu')(x)

y1 = MaxPooling1D(pool_size=2,strides=1,padding = 'same')(x)
y2 = MaxPooling1D(pool_size=5,strides=1,padding = 'same')(x)
y3 = MaxPooling1D(pool_size=11,strides=1,padding = 'same')(x)

conc = Concatenate()([x1, x2, x3, x4, x5, y1, y2, y3])

out = Conv1D(200, 10, padding = 'same',activation='elu', kernel_regularizer=regularizers.l2(1e-5),
                activity_regularizer=regularizers.l1(1e-5))(conc)
out = MaxPooling1D(pool_size=3)(out)
out = Dropout(0.1)(out)

out = Conv1D(100, 10, padding = 'same',activation='elu', kernel_regularizer=regularizers.l2(1e-5),
                activity_regularizer=regularizers.l1(1e-5))(out)
out = MaxPooling1D(pool_size=3)(out)
out = Dropout(0.1)(out)

out = Conv1D(50, 10, padding = 'same',activation='elu', kernel_regularizer=regularizers.l2(1e-5),
                activity_regularizer=regularizers.l1(1e-5))(out)
out = Dropout(0.1)(out)

fc = Flatten()(out)
fc = Dense(num_classes**2, activation='relu')(fc)
fc = Dropout(0.3)(fc)
fc = Dense(num_classes, activation='softmax')(fc)

model = Model(x,fc)

print(out.shape)

(?, 2, 50)


In [122]:
# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=3e-4)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [123]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 1774 samples, validate on 339 samples
Epoch 1/50
1774/1774 [==============================] - 125s 71ms/step - loss: 6.6149 - acc: 0.2610 - val_loss: 5.9799 - val_acc: 0.3156
Epoch 2/50
1774/1774 [==============================] - 131s 74ms/step - loss: 6.1978 - acc: 0.3579 - val_loss: 5.5871 - val_acc: 0.3245
Epoch 3/50
1774/1774 [==============================] - 125s 70ms/step - loss: 5.7752 - acc: 0.4487 - val_loss: 5.4512 - val_acc: 0.3304
Epoch 4/50
1774/1774 [==============================] - 114s 64ms/step - loss: 5.4829 - acc: 0.5462 - val_loss: 5.3566 - val_acc: 0.3215
Epoch 5/50
1774/1774 [==============================] - 120s 68ms/step - loss: 5.2382 - acc: 0.6229 - val_loss: 5.3362 - val_acc: 0.3569
Epoch 6/50
1774/1774 [==============================] - 125s 70ms/step - loss: 5.1489 - acc: 0.6623 - val_loss: 5.3375 - val_acc: 0.3304
Epoch 7/50
1774/1774 [==============================] - 127s 72ms/step - loss: 4.9375 - acc: 0.7300 - val_loss: 5.4754 - val_acc: 0

KeyboardInterrupt: 

In [124]:
predictions = model.predict(cnn_test_data_X)
# round predictions
rounded = [np.argmax(x) for x in predictions]
print(1.0*np.sum(rounded==cnn_test_data_Y)/len(cnn_test_data_Y))

0.325842696629
